In [1]:
import pydicom as dicom
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

import tensorflow as tf
from tensorflow import keras
from keras import metrics
tf.get_logger().setLevel('INFO')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-10-26 11:07:32.787818: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os 
cn_path = '/Users/adelinechin/207/CN2'
cn_path_list = os.listdir(cn_path)
cn_list = []
for path in cn_path_list:
    ds = dicom.dcmread(cn_path + '/' + path, force=True)
    try:
        # Dicom to pixel array
        pixel_array = ds.pixel_array.astype(float)
        
    except AttributeError:
        continue
    cn_list.append(pixel_array)

In [3]:
ad_path = '/Users/adelinechin/207/AD2'
ad_path_list = os.listdir(ad_path)
ad_list = []
for path in ad_path_list:
    ds = dicom.dcmread(ad_path + '/' + path, force=True)
    try:
        # Dicom to pixel array
        pixel_array = ds.pixel_array.astype(float)
    except AttributeError:
        continue
    # Dicom to pixel array
    ad_list.append(pixel_array)

In [4]:
data = cn_list+ ad_list
label_names = ['cn', 'ad']
labels = [0]*len(cn_list)
labels.extend([1]*len(ad_list))

In [ ]:
print(len(data))
print(len(labels))

In [9]:
indices = []
for i, elm in enumerate(data):
    if elm.shape != (256, 228):
        data = data[:i] + data[i+1:]
        labels = labels[:i] + labels[i+1:]
    else:
        indices.append(i)

In [10]:
print(len(data))
print(len(labels))

1409
1409


In [11]:
print(indices)

[2, 5, 7, 9, 10, 11, 12, 14, 16, 17, 19, 22, 24, 29, 31, 33, 34, 36, 38, 41, 42, 43, 44, 45, 48, 51, 52, 54, 55, 56, 60, 65, 72, 73, 74, 75, 76, 81, 82, 83, 87, 89, 92, 93, 94, 96, 97, 98, 99, 101, 102, 103, 106, 107, 108, 109, 110, 111, 112, 114, 117, 118, 121, 126, 128, 129, 136, 137, 138, 139, 141, 145, 147, 148, 150, 151, 152, 153, 154, 157, 160, 162, 163, 167, 173, 180, 181, 182, 183, 185, 187, 188, 189, 190, 194, 195, 197, 199, 200, 203, 207, 208, 209, 211, 216, 218, 219, 221, 222, 223, 226, 227, 228, 229, 230, 232, 234, 236, 239, 240, 242, 244, 245, 247, 249, 250, 252, 255, 256, 261, 262, 263, 268, 271, 274, 275, 277, 280, 281, 282, 286, 287, 289, 297, 299, 300, 301, 302, 303, 306, 308, 309, 311, 313, 315, 317, 322, 324, 325, 327, 328, 329, 330, 331, 333, 336, 337, 341, 342, 343, 344, 347, 349, 353, 356, 360, 361, 362, 365, 367, 372, 373, 375, 377, 378, 380, 382, 383, 384, 386, 389, 390, 392, 395, 396, 397, 398, 399, 401, 404, 405, 406, 407, 408, 415, 416, 420, 422, 424, 425, 42

In [12]:
new_data = []
new_labels = []
for i in indices:
    new_data.append(data[i])
    new_labels.append(labels[i])

IndexError: list index out of range

In [15]:
dcm_data = np.dstack(new_data)
dcm_data = np.rollaxis(dcm_data,-1)
print(dcm_data.shape)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 256 and the array at index 1 has size 228

In [8]:
test = []
test_labels = []
for i, elm in enumerate(data):
    if elm.shape != data[0].shape:
        data = data[:i] + data[i+1:]
        labels = labels[:i] + labels[i+1:]
        test.append(elm)
        test_labels.append(labels[i])
        

print(len(data))
print(len(test))

IndexError: list index out of range

In [ ]:
dcm_data = np.dstack(data)
dcm_data = np.rollaxis(dcm_data,-1)
print(dcm_data.shape)

In [ ]:
print(len(data))
print(len(labels))

In [ ]:
data = np.asarray(data)
labels = np.asarray(labels).astype(int)
np.random.seed(0)
indices = np.arange(data.shape[0])
shuffled_indices = np.random.permutation(indices)
data = data[shuffled_indices]
labels = labels[shuffled_indices]

In [ ]:
def build_model(learning_rate=0.01):
  """Build a multi-class logistic regression model using Keras.

  Args:
    n_classes: Number of classes in the dataset
    learning_rate: The desired learning rate for SGD.

  Returns:
    model: A tf.keras model (graph).
  """
  tf.keras.backend.clear_session()
  np.random.seed(0)
  tf.random.set_seed(0)

  model = keras.Sequential()
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(
      units=1,
      use_bias=True,
      activation="sigmoid"
  ))

  optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

  model.compile(loss="binary_crossentropy",
                optimizer=optimizer,
                metrics=[metrics.binary_accuracy])
  return model

In [ ]:
model = build_model(0.01)

history = model.fit(
  x = dcm_data,
  y = labels,
  epochs=5,
  batch_size=64,
  validation_split=0.1,
  verbose=1)

history = pd.DataFrame(history.history)
display(history)